# Large size vectors: lcc MAD-Mex

## Normalize, reproject to wgs84

In [1]:
import unicodedata

from pyproj import Proj

import fiona
from fiona.crs import to_string
from rasterio.warp import transform_geom
from rasterio.crs import CRS as CRS_rio

In [2]:
#direc = "/LUSTRE/MADMEX/products/landcoverchange/landsat/mosaics/2017_2018"
direc = "/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/HIDALGO/"

In [3]:
#input_filename = direc + "/madmex_landsat_changes_2017-2018.shp"
#output_filename = direc + "/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp"
#layer = "madmex_landsat_changes_2017-2018_wgs84_using_fiona"
input_filename = direc + "HIDALGO_merge.shp"
output_filename = direc + "HIDALGO_merge_wgs84_using_fiona2.shp"
layer = "HIDALGO_merge_wgs84_using_fiona2"
dst_crs = "4326"
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [4]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

In [5]:
def reproj_geom(geom, source_crs, destiny_crs):
    return transform_geom(CRS_rio.from_proj4(source_crs), CRS_rio.from_epsg(destiny_crs),
                          geom)

In [6]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8')

In [7]:
def wrapper_reproj_normalize_and_write_vector(feature_collection, feature_collection_schema,
                                              source_crs, destiny_crs,
                                              output_filename,
                                              is_geographic = True):
    dst = fiona.open(output_filename, 'w',
                     driver = 'ESRI Shapefile',
                     layer = layer,
                     crs = CRS_rio.from_epsg(dst_crs).to_proj4(),
                     schema = feature_collection_schema)
    dst.close()
    with fiona.open(output_filename, 'a',
                    driver = 'ESRI Shapefile',
                    layer = layer,
                    crs = CRS_rio.from_epsg(destiny_crs).to_proj4(),
                    schema = feature_collection_schema) as dst:
        for feature in feature_collection:
            if not is_geographic:
                feature['geometry'] = reproj_geom(feature['geometry'],
                                                  source_crs,
                                                  destiny_crs)
            for att in list_name_attributes:
                feature['properties'][att] = normalize_name_classes(feature['properties'][att])
            dst.write(feature)

In [8]:
with fiona.open(input_filename) as src:
    fc = (feature for feature in src if not None in [feature['properties'][att] for att in list_name_attributes] and
                                                     feature['geometry'] is not None)
    src_crs = to_string(src.crs)
    proj_src_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_src_crs.crs.is_geographic:
        wrapper_reproj_normalize_and_write_vector(fc, fc_schema,
                                                  src_crs, 
                                                  dst_crs,
                                                  output_filename,
                                                  is_geographic = False)
    else:
        wrapper_reproj_normalize_and_write_vector(fc, fc_schema, 
                                                  src_crs, dst_crs,
                                                  output_filename)
        

# Example: register large vector layer in geonode

**Make sure stack deployment of geonode is up and running**

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pass`**

In [9]:
import subprocess

In [37]:
#next need to be in lowercase
name_table = "hidalgo_madmex_lcc_2017_2018_fiona" #madmex_landsat_2017-2018_lcc


In [21]:
l_cmd1 = ["shp2pgsql",
          output_filename,
          name_table,
          "public." + name_table + ".shp"]

In [22]:
p1 = subprocess.Popen(l_cmd1, stdout=subprocess.PIPE)

In [23]:
l_cmd2 = ["psql",
          "-q",
          "-h",
          "geonode.conabio.gob.mx",
          "-d",
          "geonode_data",
          "-U", 
          "geonode"]

In [24]:
p2 = subprocess.Popen(l_cmd2, stdin=p1.stdout, stderr=subprocess.PIPE)

In [25]:
p2.communicate()

(None, b'')

In [26]:
p2.poll()

0

In [27]:
p1.terminate()

In [28]:
p2.terminate()

**2) Add to geoserver from geonode_data database**. Need to follow: https://training.geonode.geo-solutions.it/004_admin_workshop/007_loading_data_into_geonode/geoserver.html

**3) Update layers using docker container `spcgeonode_django_1`**

In [29]:
from docker import APIClient

In [30]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [31]:
c

In [43]:
cmd = "".join(["python manage.py updatelayers -s geonode_data -w geonode -f ",
               name_table, " ",
               "--settings=geonode.local_settings"])

In [50]:
cmd

'python manage.py updatelayers -s geonode_data -w geonode -f hidalgo_madmex_lcc_2017_2018_fiona --settings=geonode.local_settings'

In [46]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)

In [47]:
ex_start = c.exec_start(exec_id=ex, stream=True)

**To retrieve output:**

In [49]:
#for val in ex_start:
#    print(val)

**4) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# Example: `importlayers` for large raster

## Reproject and compress tif

In [1]:
import numpy as np

from pyproj import Proj
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'

In [3]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
dst_crs = 'EPSG:4326'

In [4]:
def wrapper_reproj_and_write(source_dataset, destiny_crs, output_filename,
                             is_geographic = True):
    source_meta = source_dataset.meta.copy()
    source_transform = source_dataset.transform
    source_width = source_dataset.width
    source_height = source_dataset.height
    source_crs = source_dataset.crs
    if not is_geographic:
        transform, width, height = calculate_default_transform(source_crs, destiny_crs, 
                                                               source_width, source_height, 
                                                               *source_dataset.bounds)
        source_meta.update({'crs': destiny_crs,
                           'transform': transform,
                           'width': width,
                           'height': height
                            })
    else:
        transform, width, height = source_transform, source_width, source_height
    
    source_meta.update({'driver': 'GTiff',
                        'count': 1,
                        'dtype': rasterio.uint8,
                        'compress': 'lzw',
                        'tiled': True
                        })
    with rasterio.open(output_filename, 'w', **source_meta,
                       ) as dst:
        array = np.zeros((height, width), dtype=rasterio.uint8)
        reproject(source=rasterio.band(source_dataset, 1),
                  destination=array,
                  src_transform=source_transform,
                  src_crs=source_crs,
                  dst_transform=transform,
                  dst_crs=dst_crs,
                  resampling=Resampling.nearest
                  )  
        dst.write(array, 1)

In [5]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        wrapper_reproj_and_write(src, dst_crs, output_filename,
                                 is_geographic=False)
    else:
        wrapper_reproj_and_write(src, dst_crs, output_filename)

# Example: register large vector layer in geonode

**1) Importlayers cmd**

In [28]:
from docker import APIClient

In [29]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [30]:
c

In [31]:
region = "".join(["\"Hidalgo, Mexico, North America, Latin America\""])
name = "\"sentinel2_Hidalgo_2017_31_tiled\""
title = "\"sentinel2_Hidalgo_2017_31_tiled\""
abstract = "\"Sentinel2 MAD-Mex land cover classification\""
kw = "\"MAD-Mex, Sentinel2, GeoTIFF, WCS\""


In [32]:
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               output_filename, " ",
               "--settings=geonode.local_settings"])

In [33]:
cmd

'python manage.py importlayers -v 3 -i -o -n "sentinel2_Hidalgo_2017_31_tiled" -t "sentinel2_Hidalgo_2017_31_tiled" -a "Sentinel2 MAD-Mex lc" -k "MAD-Mex, Sentinel2, GeoTIFF, WCS" -r "Hidalgo Mexico, North America, Latin America" /LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31//Hidalgo_2017_31_wgs84_tiled_rasterio.tif --settings=geonode.local_settings'

In [34]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)

In [35]:
ex_start = c.exec_start(exec_id=ex, stream=True)

**To retrieve output:**

In [24]:
#for val in ex_start:
#    print(val)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

## Small to Medium size vectors

In [1]:
import geopandas as gpd

## Landsat

In [2]:
direc = '/datos/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/AGUASCALIENTES'

In [6]:
input_file = direc + '/AGUASCALIENTES_merge.shp'
out_filename = direc + '/AGUASCALIENTES_merge_wgs84_using_fiona.shp'
layer = 'AGUASCALIENTES_merge_wgs84_using_fiona'

In [7]:
gdf = gpd.read_file(input_file)

In [ ]:
crs_src = gdf.crs

In [ ]:
crs_dst = "EPSG:4326"

In [ ]:
gdf_reproj = gpd.to_crs(crs_dst)